<h1><center>Toronto neighborhood segmentation
</center></h1>


## 1. Importando librerías 


In [16]:
!pip install beautifulsoup4
!pip install geocoder
!pip install pgeocode
import pandas as pd # librería para el análisis de datos
import requests # librería para gestionar las peticiones
from bs4 import BeautifulSoup # librería para parsear documentos html
import geocoder # librería para obtener localización geográfica
import pgeocode

## 2. Adquiriendo los datos 

In [17]:
# Obteniendo la respuesta en formato html
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response=requests.get(wikiurl)

In [18]:
#Convirtiendo los datos de html a un objeto beautifulsoup
soup = BeautifulSoup(response.text, 'html.parser')
#Buscamos la tabla objetivo en el objeto beautifulsoup
table = soup.find('table')
#obtenemos todas las celdas de la tabla y las guardamos en una variable
target = table.find_all('p')
#extraemos la información de las celdas
postal_codes = []
boroughs = []
neighborhoods = []

for cel in target:
    neighborhood = []
    postal_codes.append(cel.find('b').contents[0])
    if cel.find('span').find_all('a') == []:
        boroughs.append('Not assigned')
        neighborhoods.append('Not assigned')
    else:
        for item in cel.find('span').find_all('a'):
            neighborhood.append(item.contents[0])
        boroughs.append(neighborhood[0])
        neighborhood.pop(0)
        neighborhoods.append(neighborhood)
test = []
for item,x in zip(neighborhoods,boroughs):
    string = ' '
    if len(item) > 1 and isinstance(item,list):
        for x in item:
            string +=  ', ' + x
        string = string[2:]
        test.append(string)
    elif isinstance(item,list) and item != []:
        test.append(item[0])
    elif isinstance(item,list):
        test.append(x)
    else:
        test.append(item)

neighborhoods = test

In [19]:
#Creamos un dataframe con los datos obtenidos
data = {'PostalCode' : postal_codes, 
        'Borough': boroughs, 
        'Neighborhood': neighborhoods}
Toronto_neighborhoods = pd.DataFrame(data)
Toronto_neighborhoods = Toronto_neighborhoods[Toronto_neighborhoods['Borough'] != 'Not assigned'].reset_index(drop=True)
Toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


## 3. Adquiriendo latitud y longitud

In [33]:
# Inicializamos listas para almacenar las longitudes y latitudes de cada código postal
latitudes = []
longitudes = []
# Instanciamos un objeto pgeocode
query_object = pgeocode.Nominatim('CA')
# Obtenemos las longitudes y latitudes de cada código postal y las almacenamos en las listas previamente inicializadas
for code in Toronto_neighborhoods['PostalCode']:
    lat_lng_coords = query_object.query_postal_code(code)
    latitudes.append(lat_lng_coords.latitude)
    longitudes.append(lat_lng_coords.longitude)

In [32]:
# Agregamos las latitudes y longitudes al dataframe de vecindarios
Toronto_neighborhoods.insert(3,'Latitude',latitudes)
Toronto_neighborhoods.insert(4,'Longitude',longitudes)
Toronto_neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Queen's Park,43.6641,-79.3889
...,...,...,...,...,...
96,M8X,Etobicoke,"The Kingsway, Old Mill",43.6518,-79.5076
97,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
98,M7Y,Business reply mail,Business reply mail,43.7804,-79.2505
99,M8Y,Etobicoke,"Old Mill, Sunnylea, Humber Bay, Mimico, The Q...",43.6325,-79.4939
